In [1]:
from configparser import ConfigParser
import pandas as pd
import FuzzyMatch as fm
import DatafileIO as dio
import ComposeFunction as cf
import ConfigReader as cr
import DECAF

In [2]:
filepath = "tests/data/fuzzy_match_companies.xlsx"

companies = dio._load_files(filepath)

companies.head(5)

,reference_name,new_name,expected
0,Ballentine Partners LLC,"Ballentine Partners, LLC",match
1,Huron Capital Partners LLC,Huron Capital,match
2,USI Insurance Svc,Usi Insurance Services,match
3,"New Heritage Capital, LLC",New Heritage Capital,match
4,Aureus,Aureus Capital,match


In [3]:
# Args
input_file = filepath
col1 = "reference_name"
col2 = "new_name"

# Parameters
ini_file = "src/config/processing/fuzzy_match_company.ini"

# Setup - read
config = cr.read_config(ini_file)
all_fns = dio.read_functions_from_ini(ini_file)
# Setup - ini params
fns_proc = all_fns["functions"]
fns_clean = all_fns["cleaning"]
threshold_high = int(config["parameters"]["threshold_high"])
threshold_low = int(config["parameters"]["threshold_low"])
file_suffix = config["info"]["name"]

# Load & process data
data = dio.load_file(input_file)
data["clean1"] = cf.eval_functions_list(data[col1], fns_clean)
data["clean2"] = cf.eval_functions_list(data[col2], fns_clean)

In [4]:
data

,reference_name,new_name,expected,clean1,clean2
0,Ballentine Partners LLC,"Ballentine Partners, LLC",match,ballentine partners,ballentine partners
1,Huron Capital Partners LLC,Huron Capital,match,huron capital partners,huron capital
2,USI Insurance Svc,Usi Insurance Services,match,usi insurance svc,usi insurance services
3,"New Heritage Capital, LLC",New Heritage Capital,match,new heritage capital,new heritage capital
4,Aureus,Aureus Capital,match,aureus,aureus capital
5,"Marsh USA, Inc.",Marsh Inc,match,marsh usa,marsh
6,Madison Capital LLC,Madison Capital Funding,match,madison capital,madison capital funding
7,Adams Street Partners LLC,"Partners, Adams St",match,adams street partners,partners adams st
8,Promus Bank & Trust,Promus Capital LLC,new,promus bank trust,promus capital
9,Huron Consulting Group,Huron Capital Partners LLC,new,huron consulting group,huron capital partners


In [16]:
data["scores"] = fm.fuzzy_match_pairwise(data["clean1"], data["clean2"])
data[["clean1", "clean2", "scores"]]

,clean1,clean2,scores
0,ballentine partners,ballentine partners,100
1,huron capital partners,huron capital,90
2,usi insurance svc,usi insurance services,87
3,new heritage capital,new heritage capital,100
4,aureus,aureus capital,90
5,marsh usa,marsh,90
6,madison capital,madison capital funding,90
7,adams street partners,partners adams st,86
8,promus bank trust,promus capital,58
9,huron consulting group,huron capital partners,50


In [21]:
print(threshold_high, threshold_low)

data["match"] = fm.score_threshold(data["scores"], threshold_high, threshold_low)

data[["clean1", "clean2", "scores", "match"]].sort_values(by = "scores", ascending=False)

80 60


,clean1,clean2,scores,match
0,ballentine partners,ballentine partners,100,True
3,new heritage capital,new heritage capital,100,True
1,huron capital partners,huron capital,90,True
4,aureus,aureus capital,90,True
6,madison capital,madison capital funding,90,True
5,marsh usa,marsh,90,True
2,usi insurance svc,usi insurance services,87,True
7,adams street partners,partners adams st,86,True
10,comprehensive capital partners,cohesive capital planning,73,None
13,peabody partners,prospect partners,67,None


,clean1,clean2,scores,match
0,ballentine partners,ballentine partners,100,True
1,huron capital partners,huron capital,90,True
2,usi insurance svc,usi insurance services,87,True
3,new heritage capital,new heritage capital,100,True
4,aureus,aureus capital,90,True
5,marsh usa,marsh,90,True
6,madison capital,madison capital funding,90,True
7,adams street partners,partners adams st,86,True
8,promus bank trust,promus capital,58,False
9,huron consulting group,huron capital partners,50,False
